Model creation and loading

In [1]:
import torch
from pathlib import Path
from utils.models.complete_model import create_complete_model, save_complete_model, load_complete_model, save_checkpoint, load_checkpoint


# ALL PATHS
MODELS_DIR = "models/"
SEGMENTER_MODEL_PATH = f"{MODELS_DIR}dino_unet_decoder_finetuned.pth"
save_path = f"{MODELS_DIR}complete_model.pth"
checkpoint_path = f"{MODELS_DIR}model_checkpoint.pth"

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
model = create_complete_model(device=device, SEGMENTER_MODEL_PATH=SEGMENTER_MODEL_PATH)

# # Load the model
# if Path(save_path).exists():
#     model = load_complete_model(model, save_path, device=device, strict=True)

Loaded segmenter weights from models/dino_unet_decoder_finetuned.pth


Data loader creation

In [2]:
from utils.data.dataloaders import create_dataloaders

# CheXpert
CHEXPERT_DIR = "Datasets/CheXpertPlus"
chexpert_paths = {
    "chexpert_data_path": f"{CHEXPERT_DIR}/PNG",  # base PNG folder
    "chexpert_data_csv": f"{CHEXPERT_DIR}/df_chexpert_plus_240401_findings.csv",
}

# MIMIC
MIMIC_DIR = "Datasets/MIMIC"
mimic_paths = {
    "mimic_data_path": MIMIC_DIR,
    "mimic_splits_csv": f"{MIMIC_DIR}/mimic-cxr-2.0.0-split.csv.gz",
    "mimic_metadata_csv": f"{MIMIC_DIR}/mimic-cxr-2.0.0-metadata-findings-only.csv",
    "mimic_reports_path": f"{MIMIC_DIR}/cxr-record-list.csv.gz",  # must contain 'path'
    "mimic_images_dir": f"{MIMIC_DIR}/matched_images_and_masks_mimic_224/images",
}

import os
kwargs = {
    # "num_workers": os.cpu_count() // 2 if os.cpu_count() else 4,  # adjust on your VM
    # "persistent_workers": True,           # reuses workers between iterations
    # "prefetch_factor": 4,                 # each worker prefetches batches
    # "pin_memory": True,                   # if using CUDA
    # "drop_last": False
}

train_loader = create_dataloaders(
    chexpert_paths, 
    mimic_paths, 
    batch_size=4,
    split="train", 
    sampling_ratio=0.7,
    **kwargs
)

valid_loader = create_dataloaders(
    chexpert_paths,
    mimic_paths,
    batch_size=4,
    split="valid",
    sampling_ratio=0.7,
    **kwargs
)

images, findings, image_paths, _ = next(iter(train_loader))
print("Batch image tensor shape:", getattr(images, "shape", "N/A"))
print("Batch findings shape:", getattr(findings, "shape", len(findings)))
print("Batch image paths shape:", getattr(image_paths, "shape", len(image_paths)))

Filtering rows with missing PNGs...
[INFO] Kept 18870/49080 rows with existing PNGs
Filtering rows with missing PNGs...
[INFO] Kept 187/496 rows with existing PNGs
Batch image tensor shape: torch.Size([4, 3, 512, 512])
Batch findings shape: 4
Batch image paths shape: 4


Training

In [3]:
from utils.training import train, EarlyStopping, EarlyStoppingConfig
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = Adam(model.parameters(), lr=1e-5, weight_decay=0.1)
# scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-6)

epochs = (5 * len(train_loader)) // 100
from transformers import get_cosine_schedule_with_warmup
total_steps = 5 * len(train_loader)
warmup_steps = max(1, int(0.05 * total_steps))
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

early = EarlyStopping(EarlyStoppingConfig(
    patience=15, min_delta=1e-4, mode="min", restore_best=True,
    best_ckpt_path="checkpoints/model_best_chexpert.pth"
))
print(f"Current memory before training: {torch.cuda.memory_allocated(device) / 1e9:.2f} GB") if device == "cuda" else None
torch.cuda.empty_cache() if device == "cuda" else None
train(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    epochs=epochs,                       # total target; not "remaining"
    device=device,
    log_dir="runs/chestx_exp2_chexpert",       # SAME dir to keep appending
    checkpoint_path="checkpoints/model_epoch_chexpert.pth",
    validate_every=1,
    ckpt_every=2,
    scheduler=scheduler,
    scheduler_step_on="step",
    early_stopping=early,
    resume_from="checkpoints/model_best_chexpert.pth",  # or model_best.pth if you prefer to start from best weights
    # start_epoch=...,                 # optional override
    # start_global_step=...,           # optional override
)


Current memory before training: 0.85 GB
🔁 Resuming from checkpoint: checkpoints/model_best_chexpert.pth
Loaded checkpoint from checkpoints/model_best_chexpert.pth (epoch=9, global_step=909, tokens_cum=147888, best_metric=1.8471332144737245)


Epoch 10 Training:   2%|▏         | 101/4718 [01:03<48:41,  1.58batch/s] 


Epoch 10 | Loss: 4.0677 | LR: 0.0000 | Tokens: 164253


Epoch 10 Validation: 100%|██████████| 47/47 [00:08<00:00,  5.75batch/s]


Epoch 10 | Validation Loss: 1.7996 | Validation Tokens: 7481
New best model found during early stopping.
Saved checkpoint to checkpoints/model_best_chexpert.pth
Saving periodic checkpoint.
Saved checkpoint to checkpoints/model_epoch_chexpert.pth


...

Epoch 73 | Validation Loss: 1.2685 | Validation Tokens: 7481


Epoch 74 Training:   2%|▏         | 101/4718 [00:39<30:15,  2.54batch/s]


Epoch 74 | Loss: 2.9172 | LR: 0.0000 | Tokens: 1238736


Epoch 74 Validation: 100%|██████████| 47/47 [00:07<00:00,  6.54batch/s]

Epoch 74 | Validation Loss: 1.2697 | Validation Tokens: 7481
Early stopping triggered at epoch 74. Best epoch: 59 with val loss: 1.261065331697464.
🎉 Training complete.
